In [ ]:
# Importando todas as bibliotecas necessárias
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
import seaborn as sns

%matplotlib inline

### Importando o banco de dados

In [ ]:
dados = pd.read_csv('Dados.csv', sep=';')
dados = dados.drop(['Operacoes_medicoes', 'Operacoes_anual', 'Temperatura_ensaio'], axis=1) # Excluindo as variáveis de tráfego "Operacoes_medicoes" e "Operacoes_anual"
dados.shape

### Separando o conjunto de dados de entrada, X, e de saída, Y

In [ ]:
X, Y = pd.DataFrame(dados.iloc[:, :6]), pd.Series(dados.iloc[:, 6])

### Dividindo os conjuntos de dados entre 90% para Treinamento e 10% para Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
print('Shape X_train:', X_train.shape) # Imprime a matriz dos dados de entrada de treinamento
print('Shape X_test:', X_test.shape) # Imprime a matriz dos dados de saída de treinamento
print('Shape y_train:', y_train.shape) # Imprime a matriz dos dados de entrada de teste
print('Shape y_test:', y_test.shape)  # Imprime a matriz dos dados de saída de teste

### Pré-processando os dados de entrada utilizando o pré-processamento Standard Scaler

In [ ]:
scaler_x = StandardScaler()
scaler_x.fit(X_train) # Ajustando os dados para a média e desvio padrão do conjunto de teste
X_train = pd.DataFrame(scaler_x.transform(X_train), columns=X.columns) # Aplicando a transformação no conjunto de treinamento
X_test = pd.DataFrame(scaler_x.transform(X_test), columns=X.columns) # O conjunto de teste é pré-processado de acordo com o conjunto de treinamento

### Estatística descritiva do conjunto de treinamento

In [ ]:
X_train.describe().round(2)

### Estatística descritiva do conjunto de teste

In [ ]:
X_test.describe().round(2)

***A melhor arquitetura de rede para a variável Nº de operações entre remoções é:***<br>
    2 camadas ocultas com 61 e 32 neurônios, respectivamente<br>
    alpha de 0.01 <br>
    random state = 7    

# Rede Neural Artificial

In [ ]:
# A variável reg armazena os hiperparâmetros da Rede Neural Artificial
reg = MLPRegressor(hidden_layer_sizes=(61, 32), # Quantidade de camadas ocultas e seus respectivos neurônios
                activation='relu', # Função de ativação
                solver='lbfgs', # Solver utilizado para otimizar os pesos
                alpha=0.01, # Regularizador, atua evitando "overfitting"
                random_state=7, # Armazena a inicialização dos pesos, permitindo replicabilidade
                max_iter=1000) # Número máximo de iterações
reg.fit(X_train, y_train) # Treinando a RNA com o conjunto de dados de treinamento, X_train

In [ ]:
# Apresentando os resultados do Treinamento
print('R² treinamento: %.2f' %(reg.score(X_train, y_train)*100))
print('Erro Médio Absoluto: %.3f' %mean_absolute_error(y_train, reg.predict(X_train)))
print('Erro Médio Quadrático: %.3f\n' %mean_squared_error(y_train, reg.predict(X_train)))

# Apresentando os resultados do Teste
print('R² teste: %.2f' %(reg.score(X_test, y_test)*100))
print('Erro Médio Absoluto: %.3f' %mean_absolute_error(y_test, reg.predict(X_test)))
print('Erro Médio Quadrático: %.3f' %mean_squared_error(y_test, reg.predict(X_test)))

### Plotando o gráfico de dispersão do treinamento

In [ ]:
plt.style.use('seaborn')
y_train_predicted = pd.Series(reg.predict(X_train))

plt.figure(figsize=(6, 6))
plt.scatter(y_train, y_train_predicted, alpha=0.5)
plt.title('Treinamento M1', fontsize=18)
plt.xlabel('Coeficiente de atrito observado', fontsize=14)
plt.ylabel('Coeficiente de atrito estimado', fontsize=14)

b, m = polyfit(y_train, y_train_predicted, 1)
plt.plot(y_train, b + m * y_train, '-', color='k', alpha=0.5)
plt.text(0.5, 0.95, "y = %.3fx + %.3f"%(m, b), fontsize=14)

plt.tight_layout()
plt.savefig('Grafico_dispersao_treinamento_M1.png', dpi=300)
plt.show()

### Plotando o gráfico de dispersão do teste

In [ ]:
y_test_predicted = pd.Series(reg.predict(X_test))

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_test_predicted, alpha=0.5)
plt.title('Teste M1', fontsize=18)
plt.xlabel('Coeficiente de atrito observado', fontsize=14)
plt.ylabel('Coeficiente de atrito estimado', fontsize=14)

b, m = polyfit(y_test, y_test_predicted, 1)
plt.plot(y_test, b + m * y_test, '-', color='k', alpha=0.5)
plt.text(0.55, 0.9, "y = %.3fx + %.3f"%(m, b), fontsize=14)

plt.tight_layout()
plt.savefig('Grafico_dispersao_teste_M1.png', dpi=300)
plt.show()

# Aplicando à ultima medição de coeficiente de atrito (14/08/2019)

In [ ]:
# Importando os dados
novo_conjunto = pd.read_csv('Aplicacao.csv', sep=';')
novo_conjunto = novo_conjunto.drop(['Temperatura_ensaio', 'Operacoes_medicoes', 'Operacoes_anual'], axis=1)
X_novo, Y_novo = pd.DataFrame(novo_conjunto.iloc[:, :6], columns=X.columns), pd.Series(novo_conjunto.iloc[:, 6])
X_novo_normalizados = pd.DataFrame(scaler_x.transform(X_novo), columns=X.columns)
coeficiente_atrito_previsto_novo = pd.Series(reg.predict(X_novo_normalizados))

In [ ]:
coeficiente_atrito_previsto_novo.describe().round(2)

### Plotando histograma de coeficiente de atrito previstos

In [ ]:
plt.style.use('seaborn')
plt.hist(coeficiente_atrito_previsto_novo, ec='black')
plt.xlabel('Coeficiente de atrito', fontsize=14)
plt.ylabel('Frequência', fontsize=14)
plt.title('Histograma de coeficiente de atrito', fontsize=18)
plt.savefig('Histograma_de_coeficiente_de_atrito_modelo_alpha_0.01.png', dpi=300)

In [ ]:
print('Erro Médio Absoluto: %.3f' %mean_absolute_error(Y_novo, coeficiente_atrito_previsto_novo))
print('Erro Quadrático Médio: %.3f' %mean_squared_error(Y_novo, coeficiente_atrito_previsto_novo))
print('R²: %.2f' %(reg.score(X_novo_normalizados, Y_novo)*100))

### Plotando o gráfico de dispersão da validação

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(Y_novo, coeficiente_atrito_previsto_novo)
plt.title('Validação', fontsize=18)
plt.xlabel('Coeficiente de atrito observado', fontsize=14)
plt.ylabel('Coeficiente de atrito estimado', fontsize=14)

b, m = polyfit(Y_novo, coeficiente_atrito_previsto_novo, 1)
plt.plot(Y_novo, b + m * Y_novo, '-', color='k', alpha=0.5)
plt.text(0.55, 1.5, "y = %.3fx + %.3f"%(m, b), fontsize=14)

plt.tight_layout()
plt.savefig('Grafico_dispersao_aplicacao_M1.png', dpi=300)
plt.show()

### Estatística descritiva do coeficiente de atrito previsto

In [ ]:
coeficiente_atrito_previsto_novo.describe().round(2)

### Estatística descritiva do coeficiente de atrito observado

In [ ]:
Y_novo.describe().round(2)